In [2]:
print("Hello")

Hello


# install apify library

In [1]:
pip install apify-client


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------- ----------------------------- 1.0/3.9 MB 10.1 MB/s eta 0:00:01
   ---------------- ----------------------- 1.6/3.9 MB 3.8 MB/s eta 0:00:01
   --------------------- ------------------ 2.1/3.9 MB 4.3 MB/s eta 0:00:01
   -------------------------------- ------- 3.1/3.9 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 3.9/3.9 MB 4.1 MB/s eta 0:00:00


# Extract some data

In [ ]:
from apify_client import ApifyClient
import pandas as pd
from textblob import TextBlob  # For sentiment analysis

# Initialize Apify client with your API key
client = ApifyClient("YOUR_APIFY_API_KEY")

# Prepare the Actor (Twitter Scraper) input
run_input = {
    "searchQueries": ["#vape OR #vaping OR 'disposable vape' lang:en"],
    "maxTweets": 500,  # Adjust as needed
    "includeReplies": False,
    "proxyConfiguration": {"useApifyProxy": True},  # Avoid blocks
}

# Run the Twitter Scraper
print("🚀 Launching Apify Twitter Scraper...")
run = client.actor("u6ppk7XqsJQib3vUd").call(run_input=run_input)

# Fetch and process results
tweets = []
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    tweets.append({
        "text": item.get("text", ""),
        "likes": item.get("likeCount", 0),
        "retweets": item.get("retweetCount", 0),
        "username": item.get("user", {}).get("username", ""),
        "date": item.get("createdAt", ""),
        "hashtags": item.get("hashtags", []),
    })

# Convert to DataFrame and analyze
df = pd.DataFrame(tweets)
df["sentiment"] = df["text"].apply(lambda x: TextBlob(x).sentiment.polarity)

# Save to CSV
df.to_csv("vape_tweets.csv", index=False)
print(f"✅ Saved {len(df)} tweets to 'vape_tweets.csv'!")

# Optional: Print top hashtags
top_hashtags = df["hashtags"].explode().value_counts().head(10)
print("🔥 Top Vape Hashtags:\n", top_hashtags)